In [ ]:
pip install -U sagemaker
pip install seaborn

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
dataset = pd.read_csv('HotelReservationsModified.csv')
dataset

,Booking_ID,no_of_adults,no_of_children,lead_time,arrival_year,arrival_month,arrival_date,no_of_special_requests,type_of_meal_plan_Meal Plan 2,room_type_reserved_Room_Type 1,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 6,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled,label_avg_price_per_room
0,INN00001,2,0,224,2017,10,2,0,False,True,False,False,True,False,False,True,1
1,INN00002,2,0,5,2018,11,6,1,False,True,False,False,False,True,False,True,2
2,INN00003,1,0,1,2018,2,28,0,False,True,False,False,False,True,True,False,1
3,INN00004,2,0,211,2018,5,20,0,False,True,False,False,False,True,True,False,2
4,INN00005,2,0,48,2018,4,11,0,False,True,False,False,False,True,True,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,85,2018,8,3,1,False,False,True,False,False,True,False,True,3
36271,INN36272,2,0,228,2018,10,17,2,False,True,False,False,False,True,True,False,2
36272,INN36273,2,0,148,2018,7,1,2,False,True,False,False,False,True,False,True,2
36273,INN36274,2,0,63,2018,4,21,0,False,True,False,False,False,True,True,False,2


In [5]:
dataset = dataset.drop(['Booking_ID'], axis=1)

label_column = dataset['label_avg_price_per_room']

def transformLabel(label):
    return label - 1

new_label_column = label_column.apply(transformLabel)
dataset['label_avg_price_per_room'] = new_label_column

dataset

,no_of_adults,no_of_children,lead_time,arrival_year,arrival_month,arrival_date,no_of_special_requests,type_of_meal_plan_Meal Plan 2,room_type_reserved_Room_Type 1,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 6,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled,label_avg_price_per_room
0,2,0,224,2017,10,2,0,False,True,False,False,True,False,False,True,0
1,2,0,5,2018,11,6,1,False,True,False,False,False,True,False,True,1
2,1,0,1,2018,2,28,0,False,True,False,False,False,True,True,False,0
3,2,0,211,2018,5,20,0,False,True,False,False,False,True,True,False,1
4,2,0,48,2018,4,11,0,False,True,False,False,False,True,True,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,85,2018,8,3,1,False,False,True,False,False,True,False,True,2
36271,2,0,228,2018,10,17,2,False,True,False,False,False,True,True,False,1
36272,2,0,148,2018,7,1,2,False,True,False,False,False,True,False,True,1
36273,2,0,63,2018,4,21,0,False,True,False,False,False,True,True,False,1


In [6]:
X = dataset.iloc[:, :-1].values
X = np.array(X).astype('float32')
X

array([[  2.,   0., 224., ...,   0.,   0.,   1.],
       [  2.,   0.,   5., ...,   1.,   0.,   1.],
       [  1.,   0.,   1., ...,   1.,   1.,   0.],
       ...,
       [  2.,   0., 148., ...,   1.,   0.,   1.],
       [  2.,   0.,  63., ...,   1.,   1.,   0.],
       [  2.,   0., 207., ...,   0.,   0.,   1.]], dtype=float32)

In [7]:
X.shape

(36275, 15)

In [8]:
Y = dataset.iloc[:, -1].values
Y = np.array(Y).astype('float32')
Y

array([0., 1., 0., ..., 1., 1., 2.], dtype=float32)

In [9]:
Y.shape

(36275,)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [11]:
X_train.shape, X_test.shape

((25392, 15), (10883, 15))

In [12]:
Y_train.shape, Y_test.shape

((25392,), (10883,))

In [13]:
import sagemaker
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.amazon.amazon_estimator import get_image_uri
import boto3
from sagemaker import Session
import sagemaker.amazon.common as smac
import io
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [ ]:
session = sagemaker.Session()
bucket = ''
subfolder_model = 'models/hotelreservations/linear-learner'
subfolder_dataset = 'datasets/hotelreservations'
key = 'hotelreservations-train-data'
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subfolder_dataset, key)
output_location = 's3://{}/{}/output'.format(bucket, subfolder_model)

In [24]:
buffer = io.BytesIO()
smac.write_numpy_to_dense_tensor(buffer, X_train, Y_train)
buffer.seek(0)

0

In [25]:
boto3.resource('s3').Bucket(bucket).Object(os.path.join(subfolder_dataset, 'train', key)).upload_fileobj(buffer)

In [26]:
container = sagemaker.image_uris.retrieve(framework='linear-learner', region = 'us-east-1')

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [30]:
linear = sagemaker.estimator.Estimator(
    image_uri = container,
    role = role,
    instance_count = 1,
    instance_type = 'ml.m4.xlarge',
    output_path = output_location,
    sagemaker_session = session,
    use_stop_instances = True,
)

In [31]:
linear.set_hyperparameters(
    feature_dim = 15,
    predictor_type = 'multiclass_classifier',
    num_classes = 3,
    num_models = 4,
    epochs = 100,
    balance_multiclass_weights = True,
    optimizer = 'sgd'
)

In [32]:
linear.fit({'train': s3_train_data})

INFO:sagemaker:Creating training-job with name: linear-learner-2024-05-23-16-48-23-330


2024-05-23 16:48:23 Starting - Starting the training job......
2024-05-23 16:48:57 Starting - Preparing the instances for training...
2024-05-23 16:49:52 Downloading - Downloading input data...
2024-05-23 16:50:17 Downloading - Downloading the training image.........
2024-05-23 16:51:33 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[05/23/2024 16:51:55 INFO 140281041315648] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'lo

In [33]:
linear_classifier = linear.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: linear-learner-2024-05-23-16-53-19-783
INFO:sagemaker:Creating endpoint-config with name linear-learner-2024-05-23-16-53-19-783
INFO:sagemaker:Creating endpoint with name linear-learner-2024-05-23-16-53-19-783


-------!

In [34]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import json

linear_classifier.serializer = CSVSerializer()
linear_classifier.deserializers = JSONDeserializer()

In [35]:
X_test.shape

(10883, 15)

In [36]:
results = json.loads(linear_classifier.predict(X_test))
results['predictions']

[{'score': [0.0694010779261589, 0.11879972368478775, 0.811799168586731],
  'predicted_label': 2},
 {'score': [0.6697376370429993, 0.2694694995880127, 0.060792841017246246],
  'predicted_label': 0},
 {'score': [0.21703888475894928, 0.5647799968719482, 0.21818110346794128],
  'predicted_label': 1},
 {'score': [0.04357728362083435, 0.11714319884777069, 0.8392795324325562],
  'predicted_label': 2},
 {'score': [0.24420519173145294, 0.48392900824546814, 0.2718658149242401],
  'predicted_label': 1},
 {'score': [0.7896483540534973, 0.1756950169801712, 0.03465663269162178],
  'predicted_label': 0},
 {'score': [0.038601264357566833, 0.1298987716436386, 0.8314999341964722],
  'predicted_label': 2},
 {'score': [0.47690775990486145, 0.31339433789253235, 0.209697887301445],
  'predicted_label': 0},
 {'score': [0.45774534344673157, 0.40405747294425964, 0.1381971389055252],
  'predicted_label': 0},
 {'score': [0.6632397174835205, 0.2731499671936035, 0.06361034512519836],
  'predicted_label': 0},
 {'sc

In [37]:
predictions = np.array([result['predicted_label'] for result in results['predictions']])
predictions

array([2, 0, 1, ..., 2, 2, 2])

In [38]:
Y_test = np.array(Y_test).astype(int)
Y_test

array([2, 0, 1, ..., 2, 2, 2])

In [39]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [40]:
accuracy_score(Y_test, predictions)

0.5730956537719378

In [42]:
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.53      0.74      0.61      3258
           1       0.57      0.32      0.41      4006
           2       0.63      0.71      0.67      3619

    accuracy                           0.57     10883
   macro avg       0.57      0.59      0.56     10883
weighted avg       0.58      0.57      0.56     10883



In [41]:
cm = confusion_matrix(Y_test, predictions)
cm

array([[2401,  502,  355],
       [1578, 1275, 1153],
       [ 583,  475, 2561]])